In [ ]:
!pip install langchain openai langchain-openai langchain-community -q

In [1]:
import re
import pandas as pd
from tqdm import tqdm
from getpass import getpass
import polars as pl
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, TransformChain
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

## Если используете ключ от OpenAI, запустите эту ячейку 👇

In [ ]:
import os
from langchain_openai import ChatOpenAI


# os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0)

## Если используете ключ из курса, запустите эти ячейки 👇


In [2]:
!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py

--2025-05-10 15:14:12--  https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11316 (11K) [text/plain]
Saving to: ‘utils.py.5’

utils.py.5          100%[===================>]  11.05K  --.-KB/s    in 0.02s   

2025-05-10 15:14:14 (566 KB/s) - ‘utils.py.5’ saved [11316/11316]



In [3]:
from utils import ChatOpenAI
from getpass import getpass

#course_api_key= "Введите ваш API ключ, полученный в боте курса"
course_api_key = getpass(prompt='Введите ваш API ключ, полученный в боте курса')

# инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

## Задание 3.2.9 🤔 Кажется, это что-то на LLM-ском? 🧐

In [4]:
df = pl.read_csv("https://stepik.org/media/attachments/lesson/1110883/raw_texts.csv")
df.head()

raw_text
str
"""The sun was setting, casting l…"
"""Le soleil se couchait, jetant …"
"""El sol se estaba poniendo, pro…"
"""La ciudad estaba llena de vida…"
"""La ville était pleine de vie, …"


Напишем функцию, которая очистит текст от ненужных символов: `¿, ¡, £`

In [6]:
def clean_text(inputs: dict) -> dict:
    text = inputs["text"]
    
    # Удаляем символы ¿, ¡, £
    text = text.str.replace("¿", "", literal=True).str.replace("¡", "", literal=True).str.replace("£", "", literal=True)
    
    # Удаляем лишние пробелы
    # В Polars можно использовать регулярные выражения для замены нескольких пробелов на один
    text = text.str.replace(" +", " ", literal=False)
    
    # Удаляем пробелы в начале и конце
    text = text.str.strip_chars()
    
    return {"text": text}

In [7]:
clean_text({"text": df['raw_text']})

{'text': shape: (13,)
 Series: 'raw_text' [str]
 [
 	"The sun was setting, casting l…
 	"Le soleil se couchait, jetant …
 	"El sol se estaba poniendo, pro…
 	"La ciudad estaba llena de vida…
 	"La ville était pleine de vie, …
 	…
 	"In a small town nestled betwee…
 	"En un pequeño pueblo situado e…
 	"Dans un petit village niché en…
 	"В городе было множество людей,…
 	"La città era piena di vita, le…
 ]}

Будем просить у модели определять язык и имя главного персонажа и выдавать ответ в виде словаря. Для этого создадим `Output parser`, с которым вы уже познакомились в прошлых уроках.

In [9]:
# Определим схемы ответа
language_schema = ResponseSchema(
    name="language",
    description="Язык текста. Название языка на английском языке. Пример: 'Russian', 'English', 'French', 'German'."
)
main_character_schema = ResponseSchema(
    name="main_character",
    description="Имя главного персонажа в тексте на том языке, на котором написан текст."
)


response_schemas = [language_schema, main_character_schema]
output_parser = StructuredOutputParser(response_schemas=response_schemas)
format_instructions = output_parser.get_format_instructions()

Напишем шаблон промпта со своим вопросом и инструкциями по форматированию ответа. Будем передавать в этот промпт сырой текст

In [10]:
prompt = PromptTemplate(
    template="""Проанализируй следующий текст и определи язык, на котором он написан, и имя главного персонажа в тексте.

Текст: {text}

{format_instructions}""",
    input_variables=["text"],
    partial_variables={"format_instructions": format_instructions}
)

Создадим цепочку с помощью `LCEL`

In [11]:
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    output_parser=output_parser,
    verbose=True
)

/tmp/ipykernel_105843/2050997211.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [12]:
for text in tqdm(df['raw_text']):
    # Применяем цепочку к тексту
    result = chain.run({"text": text})
    # Выводим результат
    print(result)
    break # Для отладки. Уберите, когда убедитесь, что на одном примере работает

  0%|          | 0/13 [00:00<?, ?it/s]/tmp/ipykernel_105843/1191620087.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain.run({"text": text})




> Entering new LLMChain chain...
Prompt after formatting:
Проанализируй следующий текст и определи язык, на котором он написан, и имя главного персонажа в тексте.

Текст: The sun was setting, casting long shadows over the small town. Jo¿hn, a middle-aged man with a heart full of dreams, was sitting on the porch of his old house. His friends, Peter and Mary, were there with him, sharing stories of their youth. They were all from the same tow¡n, and their lives were intertwined in ways they could never have imagined. Peter, a burly man with a booming voice, was the town's blacksmith. He was a man of few words, but his actions spoke volumes. Mary, on the other hand, was the town's schoolteacher. She was a woman of great wisdom and kindness, always ready to lend a helping hand. As they s¡at there, reminiscing about the past, a sense of nostalgia washed over them. They remembered the days when they were young and full of hope, when the world seemed full of possibilities. But now, as they 

  0%|          | 0/13 [00:03<?, ?it/s]


> Finished chain.
{'language': 'English', 'main_character': 'John'}


Сохраним всё в итоговый файл. Убедитесь, что на этом этапе у вас в столбцах

- `text` - очищенный текст (без символов ¿, ¡, £)
- `language` - язык, на котором написан текст (название языка указать на английском языке)
- `main_character` - имя главного персонажа в тексте (указать на том языке, на котором и написан сам текст)

In [22]:
# Создадим новый DataFrame с очищенными текстами
# Применяем очистку текста правильным способом
df_clean = df.clone()

# Применяем очистку непосредственно к каждому тексту
cleaned_texts = []
for text in df['raw_text']:
    # Очищаем текст от специальных символов
    cleaned = text.replace("¿", "").replace("¡", "").replace("£", "")
    # Удаляем лишние пробелы с помощью регулярного выражения
    cleaned = re.sub(r' +', ' ', cleaned)
    # Удаляем пробелы в начале и конце
    cleaned = cleaned.strip()
    cleaned_texts.append(cleaned)

# Добавляем очищенные тексты как новый столбец
df_clean = df_clean.with_columns([
    pl.Series("text", cleaned_texts)
])


# Создадим пустые списки для хранения результатов анализа
languages = []
main_characters = []

# Обрабатываем каждый текст
for text in tqdm(df_clean['text']):
    # Применяем цепочку к очищенному тексту
    result = chain.run({"text": text})
    # Парсим результат
    if isinstance(result, str):
        result = output_parser.parse(result)
    
    # Добавляем результаты в списки
    languages.append(result["language"])
    main_characters.append(result["main_character"])

# Добавляем столбцы language и main_character в DataFrame
df_clean = df_clean.with_columns([
    pl.Series("language", languages),
    pl.Series("main_character", main_characters)
])

# Выводим первые несколько строк для проверки
df_clean.select(['text', 'language', 'main_character']).head()

  0%|          | 0/13 [00:00<?, ?it/s]



> Entering new LLMChain chain...
Prompt after formatting:
Проанализируй следующий текст и определи язык, на котором он написан, и имя главного персонажа в тексте.

Текст: The sun was setting, casting long shadows over the small town. John, a middle-aged man with a heart full of dreams, was sitting on the porch of his old house. His friends, Peter and Mary, were there with him, sharing stories of their youth. They were all from the same town, and their lives were intertwined in ways they could never have imagined. Peter, a burly man with a booming voice, was the town's blacksmith. He was a man of few words, but his actions spoke volumes. Mary, on the other hand, was the town's schoolteacher. She was a woman of great wisdom and kindness, always ready to lend a helping hand. As they sat there, reminiscing about the past, a sense of nostalgia washed over them. They remembered the days when they were young and full of hope, when the world seemed full of possibilities. But now, as they loo

  8%|▊         | 1/13 [00:02<00:32,  2.71s/it]


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Проанализируй следующий текст и определи язык, на котором он написан, и имя главного персонажа в тексте.

Текст: Le soleil se couchait, jetant de longues ombres sur la petite ville. Pierre, un homme d'âge moyen avec un cœur plein de rêves, était assis sur le porche de sa vieille maison. Ses amis, Jacques et Marie, étaient là avec lui, partageant des histoires de leur jeunesse. Ils étaient tous de la même ville, et leurs vies étaient entrelacées de manière qu'ils n'auraient jamais pu imaginer. Jacques, un homme robuste à la voix tonitruante, était le forgeron de la ville. Il était un homme de peu de mots, mais ses actions parlaient beaucoup. Marie, en revanche, était l'institutrice de la ville. Elle était une femme de grande sagesse et de gentillesse, toujours prête à tendre la main. Alors qu'ils étaient assis là, se remémorant le passé, un sentiment de nostalgie les envahit. Ils se souvenaient des jours où i

 15%|█▌        | 2/13 [00:03<00:20,  1.85s/it]

...

 92%|█████████▏| 12/13 [00:16<00:01,  1.27s/it]


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Проанализируй следующий текст и определи язык, на котором он написан, и имя главного персонажа в тексте.

Текст: La città era piena di vita, le strade piene del suono delle chiacchiere e del rumore dei passi. Tra la folla, un uomo di nome Giovanni si distingueva. Era un detective, noto per la sua mente acuta e le sue abilità di osservazione. I suoi compagni, Sarah e Mike, erano sempre al suo fianco, aiutandolo nelle sue indagini. Erano una squadra formidabile, risolvendo casi che sconcertavano anche gli ufficiali più esperti. Giovanni era un uomo di poche parole, ma le sue azioni parlavano molto. Viveva in una piccola città, dove tutti conoscevano tutti. I suoi migliori amici, Pietro e Maria, erano sempre al suo fianco. Erano cresciuti insieme, condividendo innumerevoli avventure e creando ricordi che sarebbero durati tutta la vita. Ma la vita nella piccola città non era sempre facile. Si trovavano di fronte

100%|██████████| 13/13 [00:17<00:00,  1.33s/it]


> Finished chain.


text,language,main_character
str,str,str
"""The sun was setting, casting l…","""English""","""John"""
"""Le soleil se couchait, jetant …","""French""","""Pierre"""
"""El sol se estaba poniendo, pro…","""Spanish""","""Carlos"""
"""La ciudad estaba llena de vida…","""Spanish""","""Juan"""
"""La ville était pleine de vie, …","""French""","""Jean"""


In [23]:
# Сохраняем результаты в CSV файл
df_clean[['text', 'language', 'main_character']].write_csv('3.2.9_solution.csv')


In [19]:
pl.Config.set_tbl_rows(20)

polars.config.Config

In [21]:
df

raw_text,text,language,main_character
str,str,str,str
"""The sun was setting, casting l…","""The sun was setting, casting l…","""English""","""John"""
"""Le soleil se couchait, jetant …","""Le soleil se couchait, jetant …","""French""","""Pierre"""
"""El sol se estaba poniendo, pro…","""El sol se estaba poniendo, pro…","""Spanish""","""Carlos"""
"""La ciudad estaba llena de vida…","""La ciudad estaba llena de vida…","""Spanish""","""Juan"""
"""La ville était pleine de vie, …","""La ville était pleine de vie, …","""French""","""Jean"""
"""Die Stadt¿ war voller Leben, i…","""Die Stadt war voller Leben, ih…","""German""","""Johann"""
"""Die Sonne g¿ing unter und warf…","""Die Sonne ging unter und warf …","""German""","""Hans"""
"""В тихом уголке старого города,…","""В тихом уголке старого города,…","""Russian""","""Анна"""
"""In a small town nestled betwee…","""In a small town nestled betwee…","""English""","""Laura"""


In [24]:
df = pl.read_csv("https://stepik.org/media/attachments/lesson/1110884/custom_math_tools.csv")

In [25]:
df

task
str
"""Периметр прямоугольника со сто…"
"""Найди сумму площадей круга с р…"
"""Переведи числа 20 и 32 из деся…"
"""Переведи числа из двоичной сис…"
"""Найди объем цилиндра с радиусо…"
"""Найди длину дуги окружности с …"
"""Переведи десятичное число 1524…"
"""Найти объем куба со стороной 5…"
"""Стороны треугольника соответст…"


In [ ]:
from math import pi, sqrt

# 2 реализованные функции, как пример
@tool
def compute_arc_length(radius, angle_in_degrees):
    """Вычисляет длину дуги по радиусу и углу в градусах"""
    return (angle_in_degrees / 360) * 2 * pi * radius


@tool
def compute_triangle_area(a, b, c):
    """Вычисляет площадь треугольника по длинам его сторон"""
    s = (a + b + c) / 2
    return sqrt(s * (s - a) * (s - b) * (s - c))

# Реализуем остальные функции
@tool
def add_numbers(num1, num2):
    """Складывает два числа"""
    return num1 + num2

@tool
def convert_meters_to_cm(num):
    """Переводит метры в сантиметры"""
    return num * 100

@tool
def convert_cubic_cm_to_liters(num):
    """Переводит кубические сантиметры в литры"""
    return num / 1000

@tool
def compute_rectangle_perimeter(a, b):
    """Вычисляет периметр прямоугольника"""
    return 2 * (a + b)

@tool
def compute_circle_area(radius):
    """Вычисляет площадь круга"""
    return pi * radius ** 2

@tool
def compute_cylinder_volume(radius, height):
    """Вычисляет объем цилиндра"""
    return pi * radius ** 2 * height

@tool
def compute_cube_volume(a):
    """Вычисляет объем куба"""
    return a ** 3

@tool
def convert_binary_to_decimal(binary_number: str):
    """Переводит число из двоичной системы счисления в десятичную систему счисления"""
    return int(binary_number, 2)

@tool
def convert_decimal_to_binary(decimal_number):
    """Переводит число из десятичной системы счисления в двоичную систему счисления"""
    return bin(decimal_number)[2:]  # [2:] отрезает префикс '0b'

@tool
def get_count_ones(number: str):
    """Находит количество единиц в двоичном представлении числа"""
    # Если передано десятичное число, сначала переводим его в двоичную систему
    if isinstance(number, int):
        binary = bin(number)[2:]
    else:
        # Если передана строка, проверяем, что это двоичное число
        try:
            int(number, 2)
            binary = number
        except ValueError:
            # Если это не двоичное число, считаем, что это десятичное и переводим
            binary = bin(int(number))[2:]
    
    return binary.count('1')

# Поместим все написанные функции в tools
tools = [
    compute_arc_length, 
    compute_triangle_area,
    add_numbers,
    convert_meters_to_cm,
    convert_cubic_cm_to_liters,
    compute_rectangle_perimeter,
    compute_circle_area,
    compute_cylinder_volume,
    compute_cube_volume,
    convert_binary_to_decimal,
    convert_decimal_to_binary,
    get_count_ones
]

In [28]:
df = pl.read_csv('math_solutions.csv')  

In [29]:
df

task,answer
str,f64
"""Периметр прямоугольника со сто…",700.0
"""Найди сумму площадей круга с р…",106.81415
"""Переведи числа 20 и 32 из деся…",3.0
"""Переведи числа из двоичной сис…",901.0
"""Найди объем цилиндра с радиусо…",9.424778
"""Найди длину дуги окружности с …",2094.395102
"""Переведи десятичное число 1524…",7.0
"""Найти объем куба со стороной 5…",1.111101e6
"""Стороны треугольника соответст…",600.0


In [30]:
df = pl.read_csv('https://stepik.org/media/attachments/lesson/1110884/questions.csv')

In [31]:
df

texts,answers
str,str
"""Человек использует только 10% …","""?"""
"""Кошки и собаки видят мир в чер…","""?"""
"""Дейтерий - это изотоп водорода…","""?"""
"""Пчёлы не могут видеть красный …","""?"""
"""Нейтроны - это элементарные ча…","""?"""
"""Земля обращается вокруг Солнца…","""?"""
"""Период полураспада урана-238 с…","""?"""
"""""Дельта"" в науке обозначает из…","""?"""
"""Антибиотики эффективны против …","""?"""


Пользователи вашего сервиса присылают вам на проверку различные факты, которые они прочитали где-то в Интернете или услышали от друзей, и хотят, чтобы вы подтвердили или опровергли его.

Создайте агента, который смог бы отвечать на вопросы из широкого спектра областей знаний от математических примеров до научных фактов. Отвечать надо в формате: правда или ложь. На некоторые вопросы, возможно, агент не сможет ответить без вашей помощи, например про преподавателей курса 🙈, поэтому надо будет использовать инструмент "human", о котором упоминали в ноутбуке урока.

Вам даётся датасет содержащий набор различных фактов, напротив которых в столбце 'answers' нужно проставить: True или False (True, а не строку: 'True').

In [38]:
from langchain.agents import load_tools, Tool, initialize_agent, AgentType
from langchain.chains import LLMMathChain
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_community.tools.human.tool import HumanInputRun
from tqdm import tqdm

import numpy as np
import re

In [78]:
# Создаем математический инструмент
llm_math = LLMMathChain.from_llm(llm=llm)

math_tool = Tool(
    name='Калькулятор',
    func=llm_math.run,
    description='Полезен для вычисления математических выражений. Используй этот инструмент, когда нужно проверить математические факты.'
)

In [79]:
# Создаем инструмент для человеческого вмешательства
human_tool = HumanInputRun()

human_input_tool = Tool(
    name="Human Input",
    description="Полезен для получения информации от человека в случаях, когда сам не можешь ответить на вопрос. Используй, если не уверен в ответе или вопрос касается специфической информации, например о преподавателях курса.",
    func=human_tool.run
)

In [80]:
# Создаем инструмент для выполнения Python кода
python_tool = PythonREPLTool()

python_repl_tool = Tool(
    name="Python REPL",
    description="Полезен для выполнения Python кода и получения результатов. Используй этот инструмент, когда нужно выполнить вычисления или проверить логические выражения.",
    func=python_tool.run
)

In [81]:
# Создаем инструмент общих знаний на основе LLM
llm_tool = Tool(
    name='Общие знания',
    func=llm.predict,
    description='Полезен для ответов на вопросы о фактах, исторических событиях, науке, культуре, географии и других областях общих знаний.'
)

In [82]:
# Объединяем все инструменты
tools = [math_tool, python_repl_tool, llm_tool, human_input_tool]

# Создаем агента
fact_checker_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

In [83]:
def check_fact(statement):
    """
    Проверяет факт и возвращает True или False
    """
    prompt = f"""
    Ты агент для проверки фактов. Оцени следующее утверждение и верни ответ True, если утверждение верно, или False, если утверждение ложно.
    
    Утверждение: {statement}
    
    Ответ должен быть только True или False, без каких-либо дополнительных пояснений.
    
    Если ты не уверен в ответе, используй доступные инструменты.
    
    ВАЖНО: Не пиши слова 'True' или 'False', а верни именно булево значение: True или False.
    
    """
    
    # Получаем ответ от агента
    response = fact_checker_agent.run(prompt)
    
    # Пытаемся извлечь True или False из ответа
    true_pattern = re.compile(r'\btrue\b', re.IGNORECASE)
    false_pattern = re.compile(r'\bfalse\b', re.IGNORECASE)
    
    if true_pattern.search(response):
        return True
    elif false_pattern.search(response):
        return False
    else:
        # Если ответ не содержит явного True/False, попробуем оценить его
        # через LLM напрямую
        direct_prompt = f"""
        На основе данного ответа, определи, является ли утверждение верным (True) или ложным (False).
        Утверждение: {statement}
        Ответ: {response}
        Верни ТОЛЬКО True или False.
        """
        final_answer = llm.predict(direct_prompt)
        
        if true_pattern.search(final_answer):
            return True
        else:
            return False

In [84]:
# Тестируем агента на одном примере
test_fact = "Земля плоская"
check_fact(test_fact)



> Entering new AgentExecutor chain...
Я должен оценить утверждение о том, что Земля плоская. Это утверждение противоречит научным фактам, которые подтверждают, что Земля имеет форму геоида. Я могу использовать свои знания, чтобы подтвердить это.

Action: Общие знания
Action Input: "Земля плоская или круглая?"
Observation: Земля имеет форму, близкую к сферической, что подтверждается множеством научных данных и наблюдений. Существует множество доказательств, подтверждающих это, включая фотографии Земли из космоса, наблюдения за горизонтом, а также явления, такие как затмения и кривизна горизонта. Научное сообщество единогласно признает, что Земля — это не плоский объект, а скорее геоид, то есть форма, близкая к сфере, с небольшими неровностями.
Thought:Я теперь знаю окончательный ответ.  
Final Answer: False

> Finished chain.


False

In [85]:
# Загружаем датасет с вопросами
df = pl.read_csv('https://stepik.org/media/attachments/lesson/1110884/questions.csv')
df.head()

texts,answers
str,str
"""Человек использует только 10% своего мозга.""","""?"""
"""Кошки и собаки видят мир в черно-белых тонах.""","""?"""
"""Дейтерий - это изотоп водорода.""","""?"""
"""Пчёлы не могут видеть красный цвет.""","""?"""
"""Нейтроны - это элементарные частицы с отрицательным зарядом.""","""?"""


In [86]:
# Создаем копию датафрейма
df_with_answers = df.clone()

# Создаем пустой список для ответов
answers = []

# Обрабатываем каждый вопрос в датасете
for question in tqdm(df['texts']):
    answer = check_fact(question)
    answers.append(answer)

# Добавляем столбец с ответами
df_with_answers = df_with_answers.with_columns([
    pl.Series("answers", answers)
])

# Показываем первые несколько строк результата
df_with_answers.head()

  0%|          | 0/25 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...
Данное утверждение о том, что человек использует только 10% своего мозга, является мифом. Научные исследования показывают, что мы используем все части нашего мозга, хотя не все области активны одновременно. 

Action: Общие знания
Action Input: "Человек использует только 10% своего мозга. Верно ли это утверждение?"
Observation: Утверждение о том, что человек использует только 10% своего мозга, является мифом. Научные исследования показывают, что практически все части мозга имеют известные функции и активируются в различных ситуациях. Даже во время простых действий, таких как разговор или движение, задействуются многие области мозга. 

Современные нейробиологические исследования с использованием методов визуализации, таких как функциональная магнитно-резонансная томография (фМРТ), показывают, что мозг работает как целостная система, и даже во время отдыха он активно обрабатывает информацию и выполняет множество функций. Таким образом, утверждение о

  4%|▍         | 1/25 [00:07<02:53,  7.22s/it]

Я теперь знаю окончательный ответ.  
Final Answer: False

> Finished chain.


> Entering new AgentExecutor chain...
Я должен оценить утверждение о том, что кошки и собаки видят мир в черно-белых тонах. Я знаю, что это не совсем верно, так как и кошки, и собаки могут видеть цвета, хотя и не так ярко, как люди. Однако, чтобы быть уверенным, я проверю это утверждение.

Action: Общие знания
Action Input: "Do cats and dogs see the world in black and white?" 
Observation: Cats and dogs do not see the world in strictly black and white, but their color vision is different from that of humans. Both species have dichromatic vision, meaning they have two types of color receptors (cones) in their eyes, compared to the three types that humans have. 

Cats and dogs can see some colors, but their color perception is limited. They are thought to see shades of blue and yellow, but they have difficulty distinguishing between reds and greens. This means that while they do not see the world in black and w

  8%|▊         | 2/25 [00:14<02:40,  6.99s/it]

...

 96%|█████████▌| 24/25 [03:38<00:33, 33.10s/it]

Я теперь знаю окончательный ответ.  
Final Answer: False

> Finished chain.


> Entering new AgentExecutor chain...
Я должен проверить факты о Пушкине, чтобы определить, верно ли утверждение о его рождении. 

Action: Общие знания
Action Input: "Когда и где родился Александр Пушкин?" 
Observation: Александр Сергеевич Пушкин родился 6 июня 1799 года в Москве, Россия.
Thought:

100%|██████████| 25/25 [03:43<00:00,  8.94s/it]

Я теперь знаю окончательный ответ.  
Final Answer: False

> Finished chain.


texts,answers
str,bool
"""Человек использует только 10% своего мозга.""",false
"""Кошки и собаки видят мир в черно-белых тонах.""",false
"""Дейтерий - это изотоп водорода.""",true
"""Пчёлы не могут видеть красный цвет.""",true
"""Нейтроны - это элементарные частицы с отрицательным зарядом.""",false


In [87]:
# Сохраняем результаты в CSV файл
df_with_answers.write_csv('fact_check_answers.csv')
print("Результаты сохранены в файл fact_check_answers.csv")

Результаты сохранены в файл fact_check_answers.csv


In [88]:
df= pl.read_csv('fact_check_answers.csv')

In [89]:
pl.Config.set_tbl_rows(25)
pl.Config.set_fmt_str_lengths(1000)

polars.config.Config

"Дельта" в науке обозначает изменение квантового состояния."	

In [90]:
df

texts,answers
str,bool
"""Человек использует только 10% своего мозга.""",false
"""Кошки и собаки видят мир в черно-белых тонах.""",false
"""Дейтерий - это изотоп водорода.""",true
"""Пчёлы не могут видеть красный цвет.""",true
"""Нейтроны - это элементарные частицы с отрицательным зарядом.""",false
"""Земля обращается вокруг Солнца по эллиптической орбите.""",true
"""Период полураспада урана-238 составляет 4,5 миллиарда лет.""",true
"""""Дельта"" в науке обозначает изменение квантового состояния.""",true
"""Антибиотики эффективны против вирусов.""",false


In [70]:
df.with_columns([
    pl.when(pl.col('texts').str.contains('Курс который вы сейчас про'))
    .then (True)
    .otherwise(False)
    .alias('answers')
])

texts,answers
str,bool
"""Человек использует только 10% своего мозга.""",false
"""Кошки и собаки видят мир в черно-белых тонах.""",false
"""Дейтерий - это изотоп водорода.""",false
"""Пчёлы не могут видеть красный цвет.""",false
"""Нейтроны - это элементарные частицы с отрицательным зарядом.""",false
"""Земля обращается вокруг Солнца по эллиптической орбите.""",false
"""Период полураспада урана-238 составляет 4,5 миллиарда лет.""",false
"""""Дельта"" в науке обозначает изменение квантового состояния.""",false
"""Антибиотики эффективны против вирусов.""",false
